# Connect to Label Studio

In [9]:
from label_studio_sdk.client import LabelStudio

ls = LabelStudio(
    base_url='http://localhost:8080',
    api_key='78258cfc11b3c9f50104774d1c004842bd06bc80',
)

# Create New Project

First we need to define a **Labeling Interface** for the new project:

In [10]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import choices

label_config = LabelInterface.create({
    'text': 'Text',
    'label': choices(['Positive', 'Negative'])
})
print(label_config)

<View>
  <Text name="text" value="$text"/>
  <Choices name="label" toName="text">
    <Choice value="Positive"/>
    <Choice value="Negative"/>
  </Choices>
</View>



You can manually create [XML configuration for the labeling interface](https://labelstud.io/tags), but `LabelInterface` class provides a convenient way to create it programmatically.

Now let's create a new project with the specified labeling interface:

In [11]:
project = ls.projects.create(
    title='Text Classification',
    label_config=label_config
)

print(f'Created project:\n{ls._client_wrapper.get_base_url()}/projects/{project.id}')

Created project:
http://localhost:8080/projects/7


# Import Tasks to the Project

In [12]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world"},
        {"text": "Hello Label Studio"},
        {"text": "What a beautiful day"},
    ]
)

ProjectsImportTasksResponse(task_count=3, annotation_count=0, predictions_count=None, duration=0.03037261962890625, file_upload_ids=[], could_be_tasks_list=False, found_formats=[], data_columns=[], prediction_count=0)

## Import Tasks with Pre-annotations

In [13]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world", "sentiment": "Positive"},
        {"text": "Goodbye Label Studio", "sentiment": "Negative"},
        {"text": "What a beautiful day", "sentiment": "Positive"},
    ],
    preannotated_from_fields=['sentiment']
)

ProjectsImportTasksResponse(task_count=3, annotation_count=0, predictions_count=None, duration=0.029715776443481445, file_upload_ids=[], could_be_tasks_list=False, found_formats=[], data_columns=[], prediction_count=3)

More customizable way to import preannotations:

In [14]:
from label_studio_sdk.label_interface.objects import PredictionValue

# this returns the same `LabelInterface` object as above
li = ls.projects.get(id=project.id).get_label_interface()

# by specifying what fields to `include` we can speed up task loading
for task in ls.tasks.list(project=project.id, include=["id"]):
    task_id = task.id
    prediction = PredictionValue(
        # tag predictions with specific model version string
        # it can help managing multiple models in Label Studio UI
        model_version='my_model_v1',
        # define your labels here
        result=[
            li.get_control('label').label(['Positive']),
        ]
    )
    ls.predictions.create(task=task_id, **prediction.model_dump())


# Export Annotations

In [15]:
for task in ls.tasks.list(project=project.id):
    print(task.id)
    print(task.annotations)
    print(task.predictions)

16
[]
[{'id': 107, 'result': [{'id': '05776213-adb7-4f9d-a31f-43cb843f6ab3', 'from_name': 'label', 'to_name': 'text', 'type': 'choices', 'value': {'choices': ['Positive']}}], 'model_version': 'my_model_v1', 'created_ago': '0\xa0minutes', 'score': 0.0, 'cluster': None, 'neighbors': None, 'mislabeling': 0.0, 'created_at': '2024-08-13T11:45:45.273976Z', 'updated_at': '2024-08-13T11:45:45.274003Z', 'model': None, 'model_run': None, 'task': 16, 'project': 7}]
17
[]
[{'id': 108, 'result': [{'id': '0e44dfd5-18cb-4544-a6df-31c0a944d36b', 'from_name': 'label', 'to_name': 'text', 'type': 'choices', 'value': {'choices': ['Positive']}}], 'model_version': 'my_model_v1', 'created_ago': '0\xa0minutes', 'score': 0.0, 'cluster': None, 'neighbors': None, 'mislabeling': 0.0, 'created_at': '2024-08-13T11:45:45.367334Z', 'updated_at': '2024-08-13T11:45:45.367367Z', 'model': None, 'model_run': None, 'task': 17, 'project': 7}]
18
[]
[{'id': 109, 'result': [{'id': '7cf85fab-c0e5-465e-a397-5b54516547b6', 'from

Read more about task, annotations and predictions format in Label Studio [documentation](https://labelstud.io/guide/export#Label-Studio-JSON-format-of-annotated-tasks).